# Target Market Analysis

## Investment Analysis

### House pricing increase compare

In [7]:
import pandas as pd

# Assuming your CSV file is named 'data.csv' and contains a column named 'date' with dates in 'YYYY-MM-DD' format
df = pd.read_csv('/Users/bichengwang/Documents/codes/python-notebook/src/finance/data/SPX GOOGLEFINANCE.csv', date_parser=pd.to_datetime)

# Now df contains your data with the 'date' column parsed as datetime objects
print(df.head())

                 Date   Close
0   1/5/1990 16:00:00  352.20
1  1/12/1990 16:00:00  339.93
2  1/19/1990 16:00:00  339.15
3  1/26/1990 16:00:00  325.80
4   2/2/1990 16:00:00  330.92


/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/735624823.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('/Users/bichengwang/Documents/codes/python-notebook/src/finance/data/SPX GOOGLEFINANCE.csv', date_parser=pd.to_datetime)


In [37]:
import numpy as np

In [44]:
df['Diff'] = df['Close'] - df['Close'].shift(1)
# 300, 310, 305 -> 
# NaN, 300, 310 -> 
# agent() -> 

def buy_and_sell(df, annual_risk_free_rate=0.06, span=4):
    # span lookback weeks
    # risk free rate
    df['Moving_Average'] = df['Close'].rolling(window=span).mean()
    df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
    df['Mark'] = df['Close'] > df['EMA']
    marks = df['Mark'].values
    curr = marks[0]
    prices = df['Close'].values
    spx_returns = (df['Close'] / prices[0]).values
    base = 1.
    share = 0.
    security_returns = []

    ### buy in strategy
    for mark, price in zip(marks, prices):
        if mark != curr:
            if not mark:
                share += base / price
                base = 0.
            else:
                base += share * price
                share = 0.
        curr = mark
        security_returns.append(base + share * price)
    ratio = (base + share * prices[-1]) / (prices[-1] / prices[0])

    # Calculate the mean returns
    security_mean_return = np.mean(security_returns)
    spx_mean_return = np.mean(spx_returns)

    # Calculate the covariance and variance of returns
    covariance = np.cov(security_returns, spx_returns)[0, 1]
    variance_spx = np.var(spx_returns)

    # Calculate beta
    beta = covariance / variance_spx

    # Calculate alpha
    alpha = security_mean_return - (beta * spx_mean_return)

    # Calculate the excess return of the security over the risk-free rate
    # Sample weekly returns of the security and risk-free rate
    annual_security_returns =  np.array([y/x for x, y in zip(security_returns[:-52], security_returns[52:])])

    # Calculate the excess return of the security over the risk-free rate
    excess_returns = annual_security_returns - annual_risk_free_rate

    # Calculate the annualized average excess return
    annualized_excess_return = np.mean(excess_returns)  # Approximate number of weeks in a year

    # Calculate the annualized standard deviation of returns
    annualized_std_dev = np.std(annual_security_returns) # Approximate number of weeks in a year

    # Calculate the annual Sharpe Ratio
    annual_sharpe_ratio = annualized_excess_return / annualized_std_dev

    def calculate_maximum_withdrawal_rate(weekly_returns, target_years, tolerance=1e-6):
        withdrawal_rate = 0.01  # Starting withdrawal rate
        while True:
            # Calculate portfolio balance after target_years with current withdrawal rate
            balance = calculate_portfolio_balance(weekly_returns, withdrawal_rate)
            if balance > 0:  # If portfolio lasts for target_years, reduce withdrawal rate
                withdrawal_rate += tolerance
            else:  # If portfolio doesn't last, return previous withdrawal rate
                return withdrawal_rate - tolerance

    return {
        'ratio':ratio, 
        'origin':(prices[-1]/prices[0]), 
        'value':security_returns[-1],
        'beta': beta, 
        'alpha': alpha,
        'annual_sharp_ratio': annual_sharpe_ratio,
    }


print(df.head(20))
print(buy_and_sell(df.iloc[12:]))


                  Date   Close  ratio  Moving_Average   Diff   Mark  \
0    1/5/1990 16:00:00  352.20    NaN             NaN    NaN  False   
1   1/12/1990 16:00:00  339.93 -12.27             NaN -12.27  False   
2   1/19/1990 16:00:00  339.15  -0.78             NaN  -0.78  False   
3   1/26/1990 16:00:00  325.80 -13.35        339.2700 -13.35  False   
4    2/2/1990 16:00:00  330.92   5.12        333.9500   5.12  False   
5    2/9/1990 16:00:00  333.62   2.70        332.3725   2.70  False   
6   2/16/1990 16:00:00  332.72  -0.90        330.7650  -0.90  False   
7   2/23/1990 16:00:00  324.15  -8.57        330.3525  -8.57  False   
8    3/2/1990 16:00:00  335.54  11.39        331.5075  11.39   True   
9    3/9/1990 16:00:00  337.93   2.39        332.5850   2.39   True   
10  3/16/1990 16:00:00  341.91   3.98        334.8825   3.98   True   
11  3/23/1990 16:00:00  337.22  -4.69        338.1500  -4.69  False   
12  3/30/1990 16:00:00  339.94   2.72        339.2500   2.72   True   
13   4

/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/2532839252.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Moving_Average'] = df['Close'].rolling(window=span).mean()
/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/2532839252.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA'] = df['Close'].ewm(span=span, adjust=False).mean()
/var/folders/ls/5qzq87350jz6pv6jwnpg6mkm0000gn/T/ipykernel_35832/2532839252.py:11: SettingWithCopyWarning: 
A value is trying to be 

AttributeError: 'Series' object has no attribute 'value'